# Colorado Bike Usage

An example of poisson regression estimating 
[Colorado's bike usage](https://data.colorado.gov/Transportation/Bicycle-and-Pedestrian-Counts-in-Colorado/q2qp-xhnj/data) at different tracking stations as a function of
1. Location
2. Calendar Year
3. Month of the year.
4. Time of the day.


## Preliminaries

### Imports

In [1]:
import calendar

import numpy as np

import matplotlib.pyplot as plt

from scipy.special import factorial
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
import scipy.optimize as optimize
import pandas as pd
from sklearn.model_selection import train_test_split

import sys
sys.path.append("../..")
from E4525_ML.poisson_regression import PoissonRegression,PoissonRegressionSGD


%matplotlib inline

### Random Seed

In [2]:
seed=56373
np.random.seed(seed)

### Data

In [3]:
raw_dir="../../raw/poisson"

In [4]:
filename=raw_dir+"/Colorado_Bike_and_Pedestrian_Count_Data.csv"

In [5]:
data=pd.read_csv(filename)
data.head()

,stationId,dataCollected,county,location,lat,long,direction,month,year,dataDate,...,hr15,hr16,hr17,hr18,hr19,hr20,hr21,hr22,hr23,Location 1
0,B00001,BIKE & PEDESTRIAN,Denver,Platte River Trail south of REI\n,39.75382,-105.01009,North,7,2010,07/29/2010 12:00:00 AM,...,38,51,28,53,60,43,20,17,2,"(39.75382, -105.01009)"
1,B00001,BIKE & PEDESTRIAN,Denver,Platte River Trail south of REI\n,39.75382,-105.01009,South,7,2010,07/29/2010 12:00:00 AM,...,55,85,77,57,50,35,17,32,14,"(39.75382, -105.01009)"
2,B00001,BIKE & PEDESTRIAN,Denver,Platte River Trail south of REI\n,39.75382,-105.01009,North,7,2010,07/30/2010 12:00:00 AM,...,46,50,20,46,46,16,12,8,3,"(39.75382, -105.01009)"
3,B00001,BIKE & PEDESTRIAN,Denver,Platte River Trail south of REI\n,39.75382,-105.01009,South,7,2010,07/30/2010 12:00:00 AM,...,67,113,79,53,42,25,10,17,7,"(39.75382, -105.01009)"
4,B00001,BIKE & PEDESTRIAN,Denver,Platte River Trail south of REI\n,39.75382,-105.01009,North,7,2010,07/31/2010 12:00:00 AM,...,98,59,61,57,33,44,22,11,4,"(39.75382, -105.01009)"


## Prepare regressors

We have 126 distinct locations ,we will treat them as a categorical variable 

In [11]:
len(data["stationId"].unique())

126

We will have a variable for each month of the year

In [12]:
data["month"].unique()

array([ 7,  8, 12,  1,  2,  3,  6,  4,  5,  9, 10, 11])

We will have a variable per year to see if there are any trends

In [13]:
data["year"].unique()

array([2010, 2011, 2012, 2013, 2014, 2009])

We will treach the counts per each our as a separate observation, and use the hour as another categorical regressor. 

We will use [pandas.melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) function to flatten the hour columns.

In [18]:
collected=pd.melt(data,id_vars=["stationId","month","year","dayOfWeek"]
        ,value_vars=['hr00', 'hr01',
       'hr02', 'hr03', 'hr04', 'hr05', 'hr06', 'hr07', 'hr08', 'hr09', 'hr10',
       'hr11', 'hr12', 'hr13', 'hr14', 'hr15', 'hr16', 'hr17', 'hr18', 'hr19',
       'hr20', 'hr21', 'hr22', 'hr23'])
collected.shape

(1016400, 6)

In [20]:
collected.head(5)

,stationId,month,year,dayOfWeek,variable,value
0,B00001,7,2010,5,hr00,26
1,B00001,7,2010,5,hr00,9
2,B00001,7,2010,6,hr00,2
3,B00001,7,2010,6,hr00,6
4,B00001,7,2010,7,hr00,3


### One hot encodings

In [21]:
collected.columns

Index(['stationId', 'month', 'year', 'dayOfWeek', 'variable', 'value'], dtype='object')

In [22]:
stationIds=pd.get_dummies(collected["stationId"])

In [23]:
hours=pd.get_dummies(collected["variable"])

In [24]:
months=pd.get_dummies(collected["month"],prefix="month")

In [25]:
years=pd.get_dummies(collected["year"],prefix="year")

In [26]:
dayOfWeek=pd.get_dummies(collected["dayOfWeek"],prefix="dayOfWeek")


In [27]:
# permet de concatenter des dataframes

reg_data=pd.concat([stationIds,hours,months,years,dayOfWeek],axis=1)
reg_data.head()


,B00001,B00002,B00003,B00004,B00005,B00006,B00007,B00008,B00009,B00011,...,year_2012,year_2013,year_2014,dayOfWeek_1,dayOfWeek_2,dayOfWeek_3,dayOfWeek_4,dayOfWeek_5,dayOfWeek_6,dayOfWeek_7
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
reg_data.shape

(1016400, 175)

We have 175 regressors and over a million observations

In [29]:
# permet de convertir un dataframe en numpy array 

X=reg_data.values
Y=collected["value"].values
X.shape,Y.shape

((1016400, 175), (1016400,))

### Train/Test split

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

### Fit Regression Model

In [31]:
model=PoissonRegression()

In [32]:
model.fit(X_train,Y_train)

(813120, 1) (813120, 175)
(813120, 1) (813120, 175)


In [33]:
Y_pred=model.predict(X_train)
Y_pred.shape

(813120, 1) (813120, 175)
(813120, 1) (813120, 175)


(813120, 1)

In [34]:
err=(Y_pred-Y_train[:,np.newaxis])**2/Y_pred
err.mean()

11.968890175517124

model is **overdispersed** even on training set  mean err*2 should be close to 1.

In [35]:
Y_pred=model.predict(X_test)

(203280, 1) (203280, 175)
(203280, 1) (203280, 175)


In [36]:
err=(Y_pred-Y_test[:,np.newaxis])**2/Y_pred

In [37]:
err.mean()

12.026058284737896

Test set does a bit worse, but not by much.

## Inspect Resutls

In [ ]:
theta=model.theta[0]
results={}
results["Intercept"]=theta[0]
for idx,column in enumerate(reg_data.columns):
    results[column]=theta[idx+1]
    print(column,results[column])

The conclusions are sort of clear:
1. people bike during the day, but not at night
2. There are more bikers in the summer
3. More people bike on weekends.


In [ ]:
h=np.arange(24)
hour_theta=np.empty(len(h))
for hour in  h:
    hour_theta[hour]=results[f'hr{hour:02}']
plt.plot(h,np.exp(hour_theta))
plt.title("Biker propersity per Hour of Day")
plt.xlabel("Hour of Day")
plt.ylabel("Propensity")

In [ ]:
m=np.arange(1,13)
month_theta=np.empty(len(m))
month_label=np.empty(len(m),str)
for month in  m:
    month_label[month-1]=calendar.month_name[month]
    month_theta[month-1]=results[f'month_{month}']
plt.plot(m,np.exp(month_theta-month_theta.min()))
t=plt.xticks(m,month_label)
plt.title("Biker propensity per month")
plt.xlabel("Month")
plt.ylabel("Propensity")

In [ ]:
d=np.arange(1,8)
day_label=np.array(["Sun","Mon","Tue","Wed","Thu","Fri","Sat"])
day_theta=np.empty(len(d))
for day in  d:
    day_theta[day-1]=results[f'dayOfWeek_{day}']
plt.plot(d,np.exp(day_theta))
t=plt.xticks(d,day_label)
plt.title("Biker Propensity per day of week")
plt.xlabel("Day of week")
plt.ylabel("Propensity")

In [ ]:

years=np.array([2009,2010,2011,2012,2013,2014])
year_theta=np.empty(len(years))
for idx,year in  enumerate(years):
    year_theta[idx]=results[f'year_{year}']
plt.plot(years,np.exp(year_theta))
#t=plt.xticks(d,day_label)
plt.title("Biker Propensity per year")
plt.xlabel("Day of week")
plt.ylabel("Propensity")

In [ ]:
s='eeee'
len(s)//2